In [1]:
from json import dumps, loads
import time, os
import threading, Queue
import ast
import unittest
import requests
import random
import logging
# chainsapce
from chainspacecontract import transaction_to_solution
# from chainspacecontract.examples.surge import contract as surge_contract
from chainspacecontract.examples import surge
# crypto
from chainspacecontract.examples.utils import setup, key_gen, pack
from chainspaceapi import ChainspaceClient

# Setup logging
logging.basicConfig(level=logging.INFO, filename="execution.log", filemode="a+",
                        format="%(asctime)-15s %(levelname)-8s %(message)s")
logging.info("Starting test_surge_bids")

# Setup variables
r = requests.get('http://10.129.6.52:4999/setup.in')
setup_str = r.text
setup_str = setup_str.split('\n')
NUM_SHARDS = int(setup_str[0])
NUM_REPLICAS = int(setup_str[1])
NUM_CLIENTS = int(setup_str[2])
logging.info("NUM_SHARDS: %s, NUM_REPLICAS: %s, NUM_CLIENTS: %s "%(NUM_SHARDS, NUM_REPLICAS, NUM_CLIENTS))
CS_HOST='10.129.6.52'
G = setup()[0]


In [2]:
global_client = ChainspaceClient(host=CS_HOST,port=5000)
init_transaction = surge.init()
init_tokens = init_transaction['transaction']['outputs']
global_client.process_transaction(init_transaction)
client_divs = surge.eq_div(NUM_CLIENTS, NUM_SHARDS)
print client_divs

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": [], "methodID": "init", "parameters": [], "outputs": ["{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken

HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
[75, 75, 75, 75]


In [3]:
# Create prosumer clients for locations 
clients = []
base_port=5000
sreps = []

for s in range(0,NUM_SHARDS):
    clients.append([])
    sreps.append(surge.SREPClient(host=CS_HOST, port=base_port+s))
    
    for c in range(0,client_divs[s]):
        clients[s].append(surge.SurgeClient(host=CS_HOST, port=base_port+s))

print clients

threads = []
for s in range(0,NUM_SHARDS):
    threads.append([])
    for c in range(0,client_divs[s]):
        client = clients[s][c]
        idx = sum(client_divs[:s]) + c
        threads[s].append(threading.Thread(target=clients[s][c].create_surge_client, args=(init_tokens[idx],)) )

for s in range(0,NUM_SHARDS):
    for c in range(0,client_divs[s]):
        threads[s][c].start()

for s in range(0,NUM_SHARDS):
    for c in range(0,client_divs[s]):
        threads[s][c].join()

[[<chainspacecontract.examples.surge.SurgeClient instance at 0x7f568d752d88>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f568d752d40>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f568d752dd0>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f568d752e18>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f568d752e60>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f568d752ea8>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f568d752ef0>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f568d752f38>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f568d752f80>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f568d752fc8>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f568d70d098>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f568d70d050>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f568d70d0e0>, <chainspac

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1 POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_734c8116bc0b4230064d6395a527a448d1204cf6f734ad138f3f1bb02531aff7"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd03569925e1896fe12654cbec46dacfa1abdae12a9f91dd3e9fa3c506ee", "92c71d002bdfafd795a52c0dd5dff8831b10fa8e62a2f7e38ffae64421b789dea7c71d002b1e7b61eb985fb56eb7ba201bed23f3f432d8d68b25f65b2839cdb459"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03569925e1896fe12654cbec46dacfa1abdae12a9f91dd3e9fa3c506ee\"}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03569925e1896fe12654cbec46dacfa1abdae12a9f91dd3e9fa3c506ee\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03569925e1896fe12654cbec46dacfa1abdae12a9f91dd3e9fa3c506ee\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_8943555522a6961aa2311c6e45c45069996280ddefc95049c282b7d31641d166"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd02d01143cebf638a49b2f818c8843ac1102903b0821b9522c652cacde4", "92c71d002baeb578c10ad745f108cd3b1ec8b370d28b7cf86ee24481ec2ea6313dc71d002b0ce5d9c93e74e474bab85ee45304fc6ed35cf1c1580a223f466ac248"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02d01143cebf638a49b2f818c8843ac1102903b0821b9522c652cacde4\"}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02d01143cebf638a49b2f818c8843ac1102903b0821b9522c652cacde4\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02d01143cebf638a49b2f818c8843ac1102903b0821b9522c652cacde4\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_8943555522a6961aa2311c6e45c45069996280ddefc95049c28

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_ca076288788b50ace4e05661c6eb6c2962b740311913a079609f9beebd057cdc"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd02c669de0bda95ab14227c0caec138338f7c4392fb14808ca80632bd0b", "92c71d002b958088d36f3e3feacd7d1c6cebd7bc8f566007b18922e84acbd5179cc71d002b11598bd0381749f976af7e4ef49bb55dac65cf344930a8c0dad1fd10"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02c669de0bda95ab14227c0caec138338f7c4392fb14808ca80632bd0b\"}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02c669de0bda95ab14227c0caec138338f7c4392fb14808ca80632bd0b\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02c669de0bda95ab14227c0caec138338f7c4392fb14808ca80632bd0b\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_ca076288788b50ace4e05661c6eb6c2962b740311913a079609

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_ecabd1e410784ecca109f1e06951ad421fd97336a4b49e6a68f16ec3ae3adb79"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd039bd469e37ee98439a90d64150d9f0c4d74a2126a80a9fda5542ea7dd", "92c71d002b7faea10b1f916cc7621eebd2be22988cce31e2c4ed4cbcff90e30342c71d002b350bb73ae8c9b830e1bd8b4f759651c82961dfa8a342c9f6772613b5"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd039bd469e37ee98439a90d64150d9f0c4d74a2126a80a9fda5542ea7dd\"}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd039bd469e37ee98439a90d64150d9f0c4d74a2126a80a9fda5542ea7dd\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd039bd469e37ee98439a90d64150d9f0c4d74a2126a80a9fda5542ea7dd\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"},

...

POST http://10.129.6.52:5003/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["3_21d76f963c5c24df2e776e2c49e5bc8092d89b00b3d791ce684b802451315b43"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd025108dbe028dee52614cb592b35d6b743e4ce783236c64499f42c5b0a", "92c71d002b9b5782f4b158adddf5e22d3d1db18b3e0bdbe49c7608098f0239d7d6c71d002b1bfec008b61449148e61c717ce1a2135cdee38c47d6a3ae2eedd8e2b"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 3, \"pub\": \"c7220292cd02c9bd025108dbe028dee52614cb592b35d6b743e4ce783236c64499f42c5b0a\"}", "{\"type\": \"VoteSlipToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd025108dbe028dee52614cb592b35d6b743e4ce783236c64499f42c5b0a\"}", "{\"type\": \"EBToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd025108dbe028dee52614cb592b35d6b743e4ce783236c64499f42c5b0a\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"3_21d76f963c5c24df2e776e2c49e5bc8092d89b00b3d791ce684

POST http://10.129.6.52:5003/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["3_6fd311abf04019745e31b8f955e0e0c9e1efdce69affc9c3af4867ec88350dcf"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd034b3aa56ec329d4bf8ae69c751439e7cf347f310e0305f8a0b5a162a8", "92c71d002b2cda00c22374fe4ac12cd8b5dc97dbd5057ba432cebfd1bd771ce30ec71d002b3b1f2b410b3ae2e28b19316b3f73a22a0c32ed3b0f7dadaa642e0af2"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 3, \"pub\": \"c7220292cd02c9bd034b3aa56ec329d4bf8ae69c751439e7cf347f310e0305f8a0b5a162a8\"}", "{\"type\": \"VoteSlipToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd034b3aa56ec329d4bf8ae69c751439e7cf347f310e0305f8a0b5a162a8\"}", "{\"type\": \"EBToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd034b3aa56ec329d4bf8ae69c751439e7cf347f310e0305f8a0b5a162a8\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"3_6fd311abf04019745e31b8f955e0e0c9e1efdce69affc9c3af4

HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u

HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u

HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u

In [4]:
# Create threads for casting srep vote

threads = []
queues = []

for s in range(0,NUM_SHARDS):
    que = Queue.Queue()
    threads.append([])
    for c in range(0,client_divs[s]):
        t = threading.Thread(target=clients[s][c].cast_srep_vote, args=(sreps[s].pub, que)) 
        threads[s].append(t)
    queues.append(que)

# Run bidding threads
for s in range(0,NUM_SHARDS):
    for c in range(0,client_divs[s]):
        threads[s][c].start()

for s in range(0,NUM_SHARDS):
    for c in range(0,client_divs[s]):
        threads[s][c].join()



POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_b0f075ff4d65e9457ed74814e6981e642cd17cfd514225971cf9e14f4fce1498"], "methodID": "cast_srep_vote", "parameters": ["92c71d002b24eeb199a25488b7570bbaa30ebf7b520ef3e39b3c82118e5802784bc71d002b6f7f9733169376c54cde528984e839d1392af209438b2b10824adbdd"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd02caa98f9cf9c9c96fd13a05e34a117c967dbeb1c13c46d8d903b0dd2e\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd02678ec842cb33a252445e3029d7b665199ba37263751192655acf02d1\", \"location\": 0}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02678ec842cb33a252445e3029d7b665199ba37263751192655acf02d1\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_b0f075ff4d65e9457ed74814e6981e642cd17cfd514225971cf9e14f4fce1498": "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02678ec842cb33a25244

{"transaction": {"inputIDs": ["0_03a9a1d3f1562e6c336164441cebf0c625fa50dd235a9615779612a7bb8d2a71"], "methodID": "cast_srep_vote", "parameters": ["92c71d002bdb75d8a4bd72dc37b6fb84e2bc41eaad53d5e657d22afa13b7ff36cfc71d002b8bbe31a06f5d8431735d1a16db772555432e7e19f16bf7d5ba480e37"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd02caa98f9cf9c9c96fd13a05e34a117c967dbeb1c13c46d8d903b0dd2e\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd03569925e1896fe12654cbec46dacfa1abdae12a9f91dd3e9fa3c506ee\", \"location\": 0}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03569925e1896fe12654cbec46dacfa1abdae12a9f91dd3e9fa3c506ee\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_03a9a1d3f1562e6c336164441cebf0c625fa50dd235a9615779612a7bb8d2a71": "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03569925e1896fe12654cbec46dacfa1abdae12a9f91dd3e9fa3c506ee\"}"}}
POST http://10.129.6.

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_71b2b3181494f9e32e0f217d720634abd87f5d06c6da05598a48ff1018970664"], "methodID": "cast_srep_vote", "parameters": ["92c71d002b224e21ad6fb89abd6477ee72e5ccaf7a1660961664994f71fa216555c71d002bbff528ae04825c0a7bc5990071070f12ae42c007d5a8ee47ceb593d0"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd02caa98f9cf9c9c96fd13a05e34a117c967dbeb1c13c46d8d903b0dd2e\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd03471066dc2bfea170850179eecf6d77c346581287b97fcb19fbc53f1e\", \"location\": 0}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03471066dc2bfea170850179eecf6d77c346581287b97fcb19fbc53f1e\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_71b2b3181494f9e32e0f217d720634abd87f5d06c6da05598a48ff1018970664": "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03471066dc2bfea17085

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1

{"transaction": {"inputIDs": ["0_ca6eeadeb50faea4e982188492030ff66ee02a6c700a113ff70306370f8b5970"], "methodID": "cast_srep_vote", "parameters": ["92c71d002b308eb1955add5abcca294084e6ba88d0a4eb903054b9fae34c6fa2e3c71d002b40428f90faf059b8fa56060745388482b0316afb36f820bf62078d36"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd02caa98f9cf9c9c96fd13a05e34a117c967dbeb1c13c46d8d903b0dd2e\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd02ea06ef3ba0eb7bf1b4a34fb9f0ca32da1020346828455926275821e7\", \"location\": 0}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02ea06ef3ba0eb7bf1b4a34fb9f0ca32da1020346828455926275821e7\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_ca6eeadeb50faea4e982188492030ff66ee02a6c700a113ff70306370f8b5970": "{\"type\": \"VoteSlipToken\"

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_d6886ff80d21e44af351cd387ea646deb842b510090dadefdf64b80e44194ebe"], "methodID": "cast_srep_vote", "parameters": ["92c71d002ba56d16fd6acc73ceac7de3bd56399957276fe9f84597596cb0692d2bc71d002ba4575ce06e86cad4f92843b22aa7ed6d175720656caa7f1bb093ca9d"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd02caa98f9cf9c9c96fd13a05e34a117c967dbeb1c13c46d8d903b0dd2e\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd037887b7bee8071d4c228d284ca35fa355c02f2ad1216b2c8b3e875903\", \"location\": 0}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd037887b7bee8071d4c228d284ca35fa355c02f2ad1216b2c8b3e875903\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_d6886ff80d21e44af351cd387ea646deb842b510090dadefdf64b80e44194ebe": "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd037887b7bee8071d4c22

...

POST http://10.129.6.52:5003/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["3_c16a644c165f99ce6957d3940d81c5ec890b2ba2c426614afb6bf44541b82a65"], "methodID": "cast_srep_vote", "parameters": ["92c71d002b5faf44cc6d34fe6da7ed9fdd4da94c835edca91162b1d735a6e850c7c71d002b2a641d8efe2e161fc114348b633335bebd201f55295cd57d4b1642a2"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd03f2977cbbd2f24254984632f376b2505431bf0326ef410c954c7c00b7\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd032e5d928c7c97846bd2c2fa8a8fa8df07d87f31d0ad1add4feb060fcf\", \"location\": 3}", "{\"type\": \"VoteSlipToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd032e5d928c7c97846bd2c2fa8a8fa8df07d87f31d0ad1add4feb060fcf\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"3_c16a644c165f99ce6957d3940d81c5ec890b2ba2c426614afb6bf44541b82a65": "{\"type\": \"VoteSlipToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd032e5d928c7c97846bd2

POST http://10.129.6.52:5003/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["3_9ede6d57b1aaa4f50de86107a39e693f7bdccce5da1c8271f3d6337f38dfc8a5"], "methodID": "cast_srep_vote", "parameters": ["92c71d002b0a016b0736d2fb954b56a87020cd02ebe8cfab40c11b1a7205a34b22c71d002b6c3d670c0614d869252404423f069777d0d77da8a604299655e3e6db"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd03f2977cbbd2f24254984632f376b2505431bf0326ef410c954c7c00b7\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd0350c23d9096252e3848675a3a1572e91eab30bf3cc53e9078a575138a\", \"location\": 3}", "{\"type\": \"VoteSlipToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd0350c23d9096252e3848675a3a1572e91eab30bf3cc53e9078a575138a\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"3_9ede6d57b1aaa4f50de86107a39e693f7bdccce5da1c8271f3d6337f38dfc8a5": "{\"type\": \"VoteSlipToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd0350c23d9096252e3848

HTTP/1.1 200 OK
 POST http://10.129.6.52:5003/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["3_666003783b6f365b7515e08c6f51a9e806d672f8f751e3a35b69d502df919a9b"], "methodID": "cast_srep_vote", "parameters": ["92c71d002b3ab1f834f178d307ec8622a51b3e9d59410bab6a3ca46e49e594f2cfc71d002b630a405f909f9d167fddf24d864b4f9ba6d46c8431c7813bb73b9d5f"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd03f2977cbbd2f24254984632f376b2505431bf0326ef410c954c7c00b7\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd035ddc23f454296efee51ca7a7d9798616ea818522a394d56aa9264c70\", \"location\": 3}", "{\"type\": \"VoteSlipToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd035ddc23f454296efee51ca7a7d9798616ea818522a394d56aa9264c70\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"3_666003783b6f365b7515e08c6f51a9e806d672f8f751e3a35b69d502df919a9b": "{\"type\": \"VoteSlipToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd035

POST http://10.129.6.52:5003/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["3_8fa5a2aecfdda2eaa7f94dbbbb08638add5e9424849e551668d1d7ffcd0f090a"], "methodID": "cast_srep_vote", "parameters": ["92c71d002bc9e6226f851d1bcb0149fd19163d8bfb6b087ece7546e2bdf232aab9c71d002bbcd561625c82464f82cfe775f2155907bae60d8eaed4b9b2b5ea578c"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd03f2977cbbd2f24254984632f376b2505431bf0326ef410c954c7c00b7\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd037fb43dbf8c91fb54e316d1f2340439b8a3274ebb82a8dd90f9a76d68\", \"location\": 3}", "{\"type\": \"VoteSlipToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd037fb43dbf8c91fb54e316d1f2340439b8a3274ebb82a8dd90f9a76d68\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"3_8fa5a2aecfdda2eaa7f94dbbbb08638add5e9424849e551668d1d7ffcd0f090a": "{\"type\": \"VoteSlipToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd037fb43dbf8c91fb54e3

HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u

In [5]:
# Create srep without threading
start = time.time()
for s in range(0,NUM_SHARDS):
    votes = []
    que = queues[s]
    while not que.empty():
#     for i in range(0,2):
        votes.append(que.get())
    client = sreps[s]
    sreps[s].create_srep_client(base_port+s, tuple(votes))

POST http://127.0.0.1:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_8994c57fcb8b72f806f083b7143dd7d368b7baedf92894a8c49a5b1e5fa083f6", "0_6952e512f43cf7d15739a59e65436e95f41edf90d847da099d75b3bebccdbfad", "0_55743e8df3b3d170b719e6da6309aaf01ad61795de4baaa45372d8ac50ff1c77", "0_1e33ce72a7c86156a6051a62d26488e0ac1e80cf1ba42430c8867ac96ee174f7", "0_3b12248567637b2909cd00b95a12ac271c449c01a143044aec6d917daf514993", "0_6e716a616eb1c859b959acd9cb82994466f52790d3c492268134f794cf4e76c0", "0_546da5bd5feef5477ae71c2f9067b1b16cf9474814cd27c93ffd46ebabfce0ac", "0_a2c7656c57c24aee72657788e83b0ebd87a9c50cf40509b79a47dcc0ba6e7d82", "0_e3be3f22d36c5713d63efa52d8a761c6b39304d6ab4998304dcc7197580814c4", "0_65c781f7e925ed92fd94514e087488927cdba6d1157972a7d15ac94d1703daad", "0_aa1b77ea8d6b6593513c376fa705297302076ddbd5d8c35b73b3365dc9ca7b37", "0_651a583b99bbf09533a6ab3a317e407c8ef4e06b3bfe1a3275b3efb959f4eb80", "0_0baa1425beeb0a31792e5bb0623012a40109510fb15496f5ff98a0d7aa32751

HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
POST http://127.0.0.1:5001/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["1_8ac0c4be53fa70ea16252eb976643cda9e1f92d058da2a417e2da3d342815041", "1_fc5074bc7ee0c2f640082851ecb1343bbd0ee1f5f1c673678f7e8951dc8cce50", "1_a899e1e5bb01a18d622b859ca652e29205c70d6d2cf8f119eb4b4372934bf8fa", "1_bc5557f626607c1de46d1fca2d10ea7deea328463f5b1729493ef943e1864697", "1_c774037f74be0472e947e33791f80a77900406bd31905b47d00a6ddefd5ae0ee", "1_a5ab2173e95d50ad6d8367a37e3d801de019067413bb303594c361ef5ec7238f", "1_17b33e029f86e28bd993f3e839c24c8c95b5a39eef0caf784a81c374f358b8f2", "1_45077166f0b1f7bf56eabf76a50ce5fc566fa6b502d73a8b744c5ee3f2fae749", "1_306bd506c920821f7b9522d20ecdbb4ea60e67df91557f8a593037b2e100b156", "1_814a7d942b42ecfc6a560e11d051320456de8dd88f164d28ee923bc5b8c0f01c", "1_9d61cdfc6eeaa52e9f9dc705bb85165da0bfb5e8601528664164bc128f0f22bc", "1_7eca4b17529148ad92048e027e36f8bedf94bc1ef38eacec9bc9726e0ad3d

HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
POST http://127.0.0.1:5002/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["2_ea730ce8887f3f933b5524d8b518a888308cf4cafc68fb82e1abe9e5cb462ddf", "2_50af233890630523a940c6b877985bdb3ef58254ee698263b3b439c736adbeb4", "2_6e4eae465c7d0d2c28f574f3bf273c844baae4b62a6e944ec9f4d72596f1e16f", "2_d55e1824f9221f8770f53a64d0f361e9a21f35b9d5e9bfd8da7d5fe6dcd1bda3", "2_e800d3f268618e14f4c086fc1c4c07a10d3a469382e3962db5b4f86df5cffffe", "2_c782d1824b6270f8fea1ccbaebcad40c254ae27e11107257109b2cad974b8296", "2_f5282920e8feef64e28be294b00480e45a940f46f30bf4582a74f360ca0f48a9", "2_62fe433aaebca4c2aabb1b73e98ff2dda470a82b8569200de49bf86f4e86dd1b", "2_280f9569d97273b66cb636a8323401636b0e9019b5309a3f0a2db8698d94405d", "2_c9f8a4c6f962ddd12bcf3a733eb311476a0c2178357ca2078cafc2197c714d8c", "2_7ee873c2fef25faba2a1862f2c68c303f616815e440a2e5bbad27284d67c1295", "2_ec9623b10b0479f94782123c8c762e09a1f8df02ce9bbc936bc363692cfb5

HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}
POST http://127.0.0.1:5003/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["3_0ca8b6f7750c2068253a9cfecd2fd09a0b918965b3c54281c89aadde90b39ade", "3_a6988023c3726aecc689f7f0185a647f8bf3e07b3682f3f6287634adc6781f61", "3_d44b4e82ce84742bce8213d2757ea22b4db4cb12e23168077bcde05b41d405c3", "3_719ff14cfda8f90be81b7f469cbc1011dd1e4d78bb6219e1032697389c83aae8", "3_490f442a3918b51721c4210595adaaea5f29ad19b8be60f184d1adb643597040", "3_a8812c61b4251753aa82e660b37657f263c3db8635dc89882102b2fd8f852d0d", "3_9a3cb21cd282f8a5d569be8361e6b5d83186a2dada50e788eda1f9ab2bf00a6c", "3_a16f9000ec16159798de68d3105f28fbd1aef86661e323fe8cad64181eb955ec", "3_ebfb064078125f8bce5b0369112d817cee2d1f7b23505a0f1176aef81e47d149", "3_1f5f0788407d56aa7ac4de2964083d5d73cc29897b9a4082de3934400e7cfaef", "3_689b971aee9abf029c18fe68f22852d3aacd450bfc6a20327f38ffa63064dfe6", "3_2a1e7849e488538eb103cd50470fa9628b6763d039d979c8f89c5ac902114

HTTP/1.1 200 OK
{u'outcome': u'accepted_t_commit', u'success': u'True'}


In [6]:
# # Create srep using threading
# threads = []
# for s in range(0,NUM_SHARDS):
#     votes = []
#     que = queues[s]
#     while not que.empty():
# #     for i in range(0,2):
#         votes.append(que.get())
#     client = sreps[s]
#     t = threading.Thread(target=sreps[s].create_srep_client, args=(base_port, tuple(votes)) )
#     threads.append(t)
    
# for t in threads:
#     t.start()
    
# for t in threads:
#     t.join()

# end = time.time()
# duration = end-start
# logging.info("Execution took "+ str(duration))
# logging.info("TPS: %s"%(NUM_CLIENTS/duration))

In [7]:
end = time.time()
duration = end-start
logging.info("Execution took "+ str(duration))
logging.info("TPS: %s"%(NUM_SHARDS/duration))